In [6]:
pip install requests pyodbc pandas


^C
Note: you may need to restart the kernel to use updated packages.


In [9]:
api_key = '7cfcd6a48fbdc9fdb589c3aece7956da'  # Replace with your TMDB API key

url = f'https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&language=en-US&page=1'
response = requests.get(url)

In [11]:
response.json()

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg',
   'genre_ids': [18, 80],
   'id': 278,
   'original_language': 'en',
   'original_title': 'The Shawshank Redemption',
   'overview': 'Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.',
   'popularity': 165.001,
   'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg',
   'release_date': '1994-09-23',
   'title': 'The Shawshank Redemption',
   'video': False,
   'vote_average': 8.7,
   'vote_count': 27298},
  {'adult': False,
   'backdrop_path': '/tmU7GeKVybMWFButWEGl2M4GeiP.jpg',
   'genre_ids': [18, 80],
   'id': 238,
   'original_language': 'en',
   'or

In [2]:
import requests
import pandas as pd
import pyodbc

# Configuration
API_KEY = '7cfcd6a48fbdc9fdb589c3aece7956da'  # Replace with your TMDB API key
SQL_SERVER = 'LAPTOP-75JKGB40'  # Replace with your SQL Server address
DATABASE = 'CB_DATABASE_VARUN'       # Replace with your database name
USERNAME = ''       # Replace with your username
PASSWORD = ''       # Replace with your password

# Step 1: Extract data from TMDb API
def extract_data(api_key):
    url = f'https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&language=en-US&page=1'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['results'][:20]  # Get top 20 movies
    else:
        print("Error fetching data from TMDb API:", response.status_code)
        return []

# Step 2: Transform the data
def transform_data(raw_data):
    # Create a DataFrame
    df = pd.DataFrame(raw_data)

    # Filter necessary columns
    df = df[['title', 'release_date', 'vote_average', 'overview']]
    df.columns = ['MovieTitle', 'ReleaseDate', 'VoteAverage', 'Overview']

    return df

# Step 3: Load data to SQL Server
def load_data_to_sql(df):
    connection_string = f'DRIVER={{SQL Server}};SERVER={SQL_SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'
    
    with pyodbc.connect(connection_string) as conn:
        cursor = conn.cursor()
        
        # Create table if it doesn't exist
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='TopMoviess' AND xtype='U')
        CREATE TABLE TopMoviess (
            MovieTitle NVARCHAR(255),
            ReleaseDate DATE,
            VoteAverage DECIMAL(3, 1),
            Overview NVARCHAR(MAX)
        )
        ''')
        
        # Insert data into the table
        for index, row in df.iterrows():
            cursor.execute('''
            INSERT INTO TopMoviess (MovieTitle, ReleaseDate, VoteAverage, Overview) 
            VALUES (?, ?, ?, ?)
            ''', row['MovieTitle'], row['ReleaseDate'], row['VoteAverage'], row['Overview'])
        
        conn.commit()

# Main ETL process
def run_etl():
    # Extract
    raw_data = extract_data(API_KEY)
    
    # Transform
    transformed_data = transform_data(raw_data)
    
    # Load
    load_data_to_sql(transformed_data)
    print("ETL process completed successfully..")


run_etl()


ETL process completed successfully..
